<a href="https://www.kaggle.com/code/jobaerislam/lung-colon?scriptVersionId=251083060" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

In [2]:

# ----- Step 1: Data Transform and Loading -----
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Replace the path below with your dataset location if needed
DATA_ROOT = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set'
full_dataset = datasets.ImageFolder(root=DATA_ROOT, transform=transform)
print("Classes:", full_dataset.classes)

Classes: ['colon_image_sets', 'lung_image_sets']


In [3]:
# ----- Step 2: Dataset Split: 70/15/15 -----
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}")


Train size: 17500, Val size: 3750, Test size: 3750


In [4]:
# ----- Step 3: Train dataset split for FL clients (3 clients) -----
client_split = [train_size // 3, train_size // 3, train_size - 2 * (train_size // 3)]
client_datasets = random_split(
    train_dataset, client_split,
    generator=torch.Generator().manual_seed(42)
)
client_loaders = [DataLoader(ds, batch_size=16, shuffle=True, num_workers=2) for ds in client_datasets]

# ----- Step 4: Val/Test Loaders -----
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

In [5]:
# ----- Step 5: Define MobileNetV2 model -----
def create_model():
    model = models.mobilenet_v2(pretrained=True)
    model.classifier[1] = nn.Linear(model.last_channel, 5)  # Adjust class count if needed
    return model

# ----- Step 6: Local training function -----
def train_local(model, dataloader, epochs=1, lr=1e-3, device='cpu'):
    model.train()
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model.state_dict()

In [6]:
# ----- Step 7: Federated averaging (FedAvg) -----
def average_weights(weights_list):
    avg_weights = {}
    for key in weights_list[0].keys():
        avg_weights[key] = torch.stack([w[key].float() for w in weights_list], 0).mean(0)
    return avg_weights


In [7]:

# ----- Step 8: Evaluation function -----
def evaluate(model, dataloader, device='cpu'):
    model.eval()
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [8]:

# ----- Step 9: Federated training loop -----
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

global_model = create_model()
global_model.to(device)

NUM_ROUNDS = 5
EPOCHS_PER_CLIENT = 1

for round_num in range(1, NUM_ROUNDS + 1):
    print(f"--- Round {round_num} ---")
    local_weights = []

    for i, dataloader in enumerate(client_loaders):
        print(f"Training client {i+1} locally...")
        local_model = create_model()
        local_model.load_state_dict(global_model.state_dict())
        local_weights.append(
            train_local(local_model, dataloader, epochs=EPOCHS_PER_CLIENT, device=device)
        )

    # Federated averaging of all client models
    averaged_weights = average_weights(local_weights)
    global_model.load_state_dict(averaged_weights)

    # Evaluate on validation and test sets
    val_acc = evaluate(global_model, val_loader, device=device)
    test_acc = evaluate(global_model, test_loader, device=device)

    print(f"Round {round_num} completed.")
    print(f"Validation Accuracy: {val_acc:.4f}")
    print(f"Test Accuracy:       {test_acc:.4f}\n")

# ----- Step 10: Save the global model -----
torch.save(global_model.state_dict(), 'federated_mobilenetv2.pth')

Using device: cpu


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 107MB/s] 


--- Round 1 ---
Training client 1 locally...
Training client 2 locally...
Training client 3 locally...
Round 1 completed.
Validation Accuracy: 0.9587
Test Accuracy:       0.9600

--- Round 2 ---
Training client 1 locally...
Training client 2 locally...
Training client 3 locally...
Round 2 completed.
Validation Accuracy: 0.9923
Test Accuracy:       0.9885

--- Round 3 ---
Training client 1 locally...
Training client 2 locally...
Training client 3 locally...
Round 3 completed.
Validation Accuracy: 0.9384
Test Accuracy:       0.9408

--- Round 4 ---
Training client 1 locally...
Training client 2 locally...
Training client 3 locally...
Round 4 completed.
Validation Accuracy: 0.9856
Test Accuracy:       0.9872

--- Round 5 ---
Training client 1 locally...
Training client 2 locally...
Training client 3 locally...
Round 5 completed.
Validation Accuracy: 0.9989
Test Accuracy:       0.9997

